Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
print(os.getcwd())

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Hyperparameter Setting

In [ ]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE':128,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':41
}

Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed) #random모듈을 사용한 난수 생성 씨드를 무작위로 받아옴
    os.environ['PYTHONHASHSEED'] = str(seed) # os.environ은 os모듈에서 환경변수를 딕셔너리 형태로 제공해주는 코드
    # 환경변수란 프로그램이 실행될 때 운영체제에서 설정해주는 변수 값을 의미
    np.random.seed(seed) 
    torch.manual_seed(seed) # 학습시에 씨드가 고정되도록 씨드를 고정해준다. 위에서 한번 섞어주고 우리가 코드를 돌릴 때 마다 랜덤해질 수 있으므로 고정시켜줌
    torch.cuda.manual_seed(seed) #마찬가지로 gpu를 사용해 만들어 지는 결과들의 값을 고정해줌 
    torch.backends.cudnn.deterministic = True #결정된 알고리즘만 사용하도록 고정
    torch.backends.cudnn.benchmark = True #cudnn의 benchmark를 통해 최적의 backend 연산을 찾는 flag를 사용한다는 의미 

seed_everything(CFG['SEED']) # Seed 고정

씨드를 섞어주어서 난수를 생성한뒤 그에 따른 최적의 알고리즘으로 고정.

Data Load

In [ ]:
df = pd.read_csv('./gdrive/MyDrive/open/train.csv')

Train / Validation Split

In [ ]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])

train, validation로 데이터셋을 나누는 단계 

from sklearn.model_selection import train_test_split 모듈을 사용하여 val의 셋의 크기가 0.2가 되도록 나눔


CustomDataset

In [ ]:
# 파이토치에서 데이터셋을 쉽게 다룰 수 있도록 하는 도구로서 torch.utils.data.Dataset을 상속받아서 직접 커스텀 데이터셋을 만듦
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list):
        self.video_path_list = video_path_list
        self.label_list = label_list
        #생성자로 매개변수 생성
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        #비디오와 라벨을 받아옴
    def __len__(self):
        return len(self.video_path_list)
        #데이터셋의 길이
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['VIDEO_LENGTH']): # 비디오 길이인 10프레임 5초만큼의 길이만큼 동영상을 나누어줌 한 영상당 50개의 이미지로 분할하여 읽어옴
            _, img = cap.read()
            img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE'])) # 학습시키는 이미지의 사이즈가 동일해야 하기 때문에 사이즈 고정, 동영상은 conv하는데 오래걸리므로 적절한 사이즈 설정이 중요
            img = img / 255.
            frames.append(img)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)
        #데이터셋에서 임의의 샘플1개를 가져오는 부분 permute로 차원의 순서를 바꾸어서 return해준다 

In [ ]:
train_dataset = CustomDataset(train['video_path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['video_path'].values, val['label'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

train 데이터셋과 validation데이터셋을 불러옴

Model Define

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=13):
        super(BaseModel, self).__init__()
        self.feature_extract = nn.Sequential(
            nn.Conv3d(3, 8, (1, 3, 3)), #동영상이므로 conv3d를 해서 적용 input, output, kernel size(3차원)
            nn.ReLU(),
            nn.BatchNorm3d(8), # 마찬가지로 batchnormalization 3d 
            nn.MaxPool3d(2),
            nn.Conv3d(8, 32, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(32),
            nn.MaxPool3d(2),
            nn.Conv3d(32, 64, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(64),
            nn.MaxPool3d(2),
            nn.Conv3d(64, 128, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(128),
            nn.MaxPool3d((3, 7, 7)), 
        )
        self.classifier = nn.Linear(1024, num_classes) #구분해야 하는 클래스가 13개이므로 최종 output을 13개로 설정
        
    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x
        #위의 생성자 정의에서 정의한 함수들 forward계산 

Train

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device) #모델을 device로 올려서 cuda로 돌림
    criterion = nn.CrossEntropyLoss().to(device) #loss계산도 cuda로 할거임
    
    best_val_score = 0 #오차가 0이 나올때까지 
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1): #(1~ 11) 즉 1부터 10까지
        model.train()
        train_loss = [] #loss의 보관 장소, 나중에 시각화를 하기 위함
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad() # 기울기 초기화
            
            output = model(videos) 
            loss = criterion(output, labels)
            
            loss.backward() # back propagation이 모든 과정에서 자동화됨
            optimizer.step() #그 back propagation을 adam이란 우리가 설정한 optimizer로 자동 update해줌
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]') # trainloss와 val loss, val score를 소수점 5자라까지 프린트
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad(): # auto grad 엔진 끄기, 더이상 기울기 트래킹 안함
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist() #logit tensor에 있는 최대값 인덱스를 반환하여 numpy로 바꾸기 위해 cpu에 올리고 numpy로 바꾼뒤 list화 해주는 코드
            trues += labels.detach().cpu().numpy().tolist() #gpu메모리에 올려져서 연산된 labels의 tensor를 numpy로 바꾸기 위해 cpu에 올리고 numpy바꾼뒤 list화 해주는 코드
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

Run!!

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"]) #adam optimizer로 learning rate찾아주도록 함 초기 lr 3e-4
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Inference

In [ ]:
test = pd.read_csv('./gdrive/MyDrive/open/test.csv')

In [ ]:
test_dataset = CustomDataset(test['video_path'].values, None) #test 데이터셋을 위에 만든 CustomDataset클래스에서 불러와서 처리, 즉 동영상처리
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0) #불러온 테스트 데이터셋 중에 일부만 batch

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [ ]:
preds = inference(model, test_loader, device)

Submission -> csv로 만드는 작업

In [ ]:
submit = pd.read_csv('./gdrive/MyDrive/open/sample_submission.csv')

In [ ]:
submit['label'] = preds
submit.head()

In [ ]:
submit.to_csv('./gdrive/MyDrive/open/baseline_submit.csv', index=False)